In [ ]:
import ee
import geemap
import joblib
import numpy as np
import os
import threading
from threading import Lock

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Load RandomForest model
model = joblib.load('models/cacu/clorofila/model.joblib')
scaler = joblib.load('models/cacu/clorofila/scaler.joblib')

# Print model feature importances for debugging
print("\nModel feature importances:")
feature_names = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11', 'NDCI', 'NDVI', 'FAI', 
                'B3_B2_ratio', 'B4_B3_ratio', 'B5_B4_ratio', 'Month', 'Season']
for name, importance in zip(feature_names, model.feature_importances_):
    print(f"{name}: {importance}")

aoi = ee.Geometry.Polygon([[[-51.163395, -18.742186], [-51.163395, -18.534308], 
                           [-50.957458, -18.534308], [-50.957458, -18.742186], 
                           [-51.163395, -18.742186]]])

# Sentinel-2 collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2020-01-01', '2020-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Get collection size and print for debugging
print(f"Number of images in collection: {sentinel2.size().getInfo()}")

# Get the median image and its date
image_list = sentinel2.toList(sentinel2.size())
middle_index = ee.Number(sentinel2.size()).divide(2).floor()
middle_image = ee.Image(image_list.get(middle_index))
middle_date = ee.Date(middle_image.get('system:time_start'))

# Get the month and season from the middle image
month_value = middle_date.get('month')
season_value = ee.Number(month_value).add(2).divide(3).floor().add(1)

# Get the median image and clip it
image = sentinel2.median().clip(aoi)

# Select bands of interest
bands = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11']
image = image.select(bands)

# Sample values for debugging
sample_point = aoi.centroid()
sample_values = image.sample(sample_point, 30).first().getInfo()
print("\nSample band values at center point:")
for band in bands:
    print(f"{band}: {sample_values['properties'][band]}")

# Apply water mask using MNDWI index 
MNDWI = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
water_mask = MNDWI.gt(0.3)
image = image.updateMask(water_mask)

# Calculate indices
NDCI = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
NDVI = image.normalizedDifference(['B8', 'B4']).rename('NDVI') 
FAI = image.expression(
    'NIR - (RED + (SWIR - RED) * (NIR_wl - RED_wl) / (SWIR_wl - RED_wl))',
    {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'SWIR': image.select('B11'),
        'NIR_wl': 842,
        'RED_wl': 665,
        'SWIR_wl': 1610
    }
).rename('FAI')

# Calculate band ratios
B3_B2_ratio = image.select('B3').divide(image.select('B2')).rename('B3_B2_ratio')
B4_B3_ratio = image.select('B4').divide(image.select('B3')).rename('B4_B3_ratio')
B5_B4_ratio = image.select('B5').divide(image.select('B4')).rename('B5_B4_ratio')

# Add month and season as constant images
month = ee.Image.constant(month_value).rename('Month')
season = ee.Image.constant(season_value).rename('Season')

# Create an image with all features
image_with_indices = image.addBands([NDCI, NDVI, FAI, B3_B2_ratio, B4_B3_ratio, 
                                   B5_B4_ratio, month, season])

# Sample values after indices for debugging
sample_values_indices = image_with_indices.sample(sample_point, 30).first().getInfo()
print("\nSample values after indices calculation:")
for name in feature_names:
    print(f"{name}: {sample_values_indices['properties'][name]}")

# Create scaled bands
scaled_bands = []
for i, name in enumerate(feature_names):
    scaled_band = image_with_indices.select(name) \
        .subtract(ee.Number(scaler.mean_[i])) \
        .divide(ee.Number(scaler.scale_[i])) \
        .rename(f'scaled_{name}')
    scaled_bands.append(scaled_band)

# Combine all scaled bands
scaled_image = ee.Image.cat(scaled_bands)

# Sample scaled values for debugging
sample_scaled = scaled_image.sample(sample_point, 30).first().getInfo()
print("\nSample scaled values:")
for name in feature_names:
    print(f"scaled_{name}: {sample_scaled['properties'][f'scaled_{name}']}")

# Create prediction using RF model - we'll use simple feature weighting for this example
# since we can't directly translate the full RF structure to EE
weighted_bands = []
for i, (name, importance) in enumerate(zip(feature_names, model.feature_importances_)):
    weighted_band = scaled_image.select(f'scaled_{name}').multiply(ee.Number(importance))
    weighted_bands.append(weighted_band)

predicted_image = ee.Image.cat(weighted_bands).reduce(ee.Reducer.sum()).rename('chlorophyll_pred')

# Apply the water mask to the predicted image
predicted_image = predicted_image.updateMask(water_mask)

# Calculate min and max values
min_max_values = predicted_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

min_value = min_max_values['chlorophyll_pred_min']  
max_value = min_max_values['chlorophyll_pred_max']

print(f"\nMinimum Chlorophyll Value: {min_value}")
print(f"Maximum Chlorophyll Value: {max_value}")

# Display the predicted image on the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

# Visualization parameters
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': [
        'blue', 'cyan', 'green', 'yellow', 'orange', 'red',  
        'darkred', 'purple', 'magenta', 'brown', 'black'
    ]
}

Map.addLayer(predicted_image, vis_params, 'Predicted Chlorophyll')
Map.addLayer(aoi, {'color': 'white', 'width': 2, 'fillColor': 'transparent'}, 'AOI Boundary')
Map.addLayerControl()

def add_legend(map_obj, title, palette, min_value, max_value):
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span>{min_value:.2f}</span>
            <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span>{max_value:.2f}</span>
        </div>
    </div>
    """
    map_obj.add_html(legend_html)

add_legend(Map, 'Predicted Chlorophyll', vis_params['palette'], min_value, max_value)  

Map

FileNotFoundError: [Errno 2] No such file or directory: 'models/cacu/clorofila/RandomForest/model.joblib'